In [1]:
# https://thebook.io/080228/part04/ch14/04-01/
# 와인의 종류 예측하기: 학습의 자동 중단

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
 
import pandas as pd
import numpy
import tensorflow as tf

In [2]:
# seed 값 설정
numpy.random.seed(3)
tf.random.set_seed(3)

In [3]:
df_pre = pd.read_csv('data/wine.csv', header=None)
df = df_pre.sample(frac=0.15)

# 1쓰면 전체 다 갖고 오는거 = 하나 마나인데, 일단 함 -> 셈플을 렌덤으로 가져옴
# 원래 데이터가 섞여서 들어오는 건 있음

# 전체 데이터중 70%만 쓰겠다 -> 0.7을 주면 -> 무작위로 섞어서 70%만 씀

In [4]:
print(df.head())

        0      1     2     3      4     5      6        7     8     9      10  \
6245   5.9  0.180  0.28   1.0  0.037  24.0   88.0  0.99094  3.29  0.55  10.65   
486   10.2  0.670  0.39   1.9  0.054   6.0   17.0  0.99760  3.17  0.47  10.00   
273    8.4  0.715  0.20   2.4  0.076  10.0   38.0  0.99735  3.31  0.64   9.40   
1993   6.8  0.370  0.51  11.8  0.044  62.0  163.0  0.99760  3.19  0.44   8.80   
885    8.9  0.750  0.14   2.5  0.086   9.0   30.0  0.99824  3.34  0.64  10.50   

      11  12  
6245   7   0  
486    5   1  
273    5   1  
1993   5   0  
885    5   1  


In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 975 entries, 6245 to 2308
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       975 non-null    float64
 1   1       975 non-null    float64
 2   2       975 non-null    float64
 3   3       975 non-null    float64
 4   4       975 non-null    float64
 5   5       975 non-null    float64
 6   6       975 non-null    float64
 7   7       975 non-null    float64
 8   8       975 non-null    float64
 9   9       975 non-null    float64
 10  10      975 non-null    float64
 11  11      975 non-null    int64  
 12  12      975 non-null    int64  
dtypes: float64(11), int64(2)
memory usage: 106.6 KB
None


In [6]:
dataset = df.values
print(dataset.dtype)

# 전체가 다 숫자(float)이니까 .astype 안해도 될듯

float64


In [7]:
dataset = df.values
X = dataset[:,0:12]
Y = dataset[:,12]

In [8]:
model = Sequential()
model.add(Dense(30, input_dim=12, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
## 1이냐, 0이냐 맞추는 문제 = 2진 분류 = binary~~ -> sigmoid로 0,1이냐 뽑아주는것

#이부분 추가
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                390       
_________________________________________________________________
dense_1 (Dense)              (None, 12)                372       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 875
Trainable params: 875
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 학습 자동 중단 설정
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)

In [ ]:
# 모델 실행
model.fit(X, Y, validation_split=0.2, epochs=2000, batch_size=500, callbacks=[early_stopping_callback])

In [10]:
# 결과 출력
print("\n Accuracy: %.4f" % (model.evaluate(X, Y)[1]))

In [ ]:
# 수업 안듣고 책에있는 내용만 복사.
# 안될 수 있음 (실행 안해봄)

In [ ]:
# https://thebook.io/080228/part04/ch14/04-03/

# 와인의 종류 예측하기: 전체 코드

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
 
import pandas as pd
import numpy
import os
import tensorflow as tf
  
# seed 값 설정
numpy.random.seed(3)
tf.random.set_seed(3)

df_pre = pd.read_csv('../dataset/wine.csv', header=None)
df = df_pre.sample(frac=0.15)
dataset = df.values
X = dataset[:,0:12]
Y = dataset[:,12]
 
model = Sequential()
model.add(Dense(30, input_dim=12, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  
# 모델 저장 폴더 만들기
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
   os.mkdir(MODEL_DIR)
 
modelpath="./model/{epoch:02d}-{val_loss:.4f}.hdf5"
  
# 모델 업데이트 및 저장
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
  
# 학습 자동 중단 설정
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)
 
model.fit(X, Y, validation_split=0.2, epochs=3500, batch_size=500, verbose=0, callbacks=[early_stopping_callback,checkpointer])